In [ ]:
import cv2
import os
import argparse
import matplotlib.pyplot as plt
import os
from time import time
from sklearn import metrics
import Dataset as dataset
import bag_of_visual_words as bovw
import mpeg7_color_layout as mpeg7
import candidate_selection as cand_selec
import constraint_creation as gen_cons
import sift as SIFT
import PC_Kmeans as PCK
import COP_Kmeans as COPK
import cluster_evaluation as clus_eval
from sklearn.metrics import silhouette_score,calinski_harabasz_score,rand_score

# --------Acepting Input----------
# Initialize parser
parser = argparse.ArgumentParser()
# Adding arguments
parser.add_argument("--k", required=True)
parser.add_argument("--query", required=True)
# ---------------------------------

# -------------- Parameter Declaration -------------------
img_folder_path = '/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/'
img_class_path = '/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/ImageSets/Main/processed_files/'
img_class_set_names = [{'name': 'aeroplane', 'type': 'vehicle'},
                       {'name': 'bicycle', 'type': 'vehicle'},
                       {'name': 'bird', 'type': 'animal'},
                       {'name': 'boat', 'type': 'vehicle'},
                       {'name': 'bottle', 'type': 'furniture'},
                       {'name': 'bus', 'type': 'vehicle'},
                       {'name': 'car', 'type': 'vehicle'},
                       {'name': 'cat', 'type': 'animal'},
                       {'name': 'chair', 'type': 'furniture'},
                       {'name': 'cow', 'type': 'animal'},
                       {'name': 'diningtable', 'type': 'furniture'},
                       {'name': 'dog', 'type': 'animal'},
                       {'name': 'horse', 'type': 'animal'},
                       {'name': 'motorbike', 'type': 'vehicle'},
                       {'name': 'person', 'type': 'person'},
                       {'name': 'pottedplant', 'type': 'furniture'},
                       {'name': 'sheep', 'type': 'animal'},
                       {'name': 'sofa', 'type': 'furniture'},
                       {'name': 'train', 'type': 'vehicle'},
                       {'name': 'tvmonitor', 'type': 'electronics'}]
# BOVW parameters
bovw_kmeans_model = None
tam_patch = (15, 15)
n_patches = 250
random_state = 1
n_dic = 50  # size of the dictionary

In [ ]:
def gen_clus(c_name, cand_img, cons, f_name):
    '''Generate and Print Constrainted Cluster\n
       Parameters
           c_name: cluster name
           cand_img: candidate image list
           cons: Constraint class object
           f_name: feature name
    '''
    if c_name == "COP":
        '''---------------- COP-Kmeans ---------------'''
        c_kmeans = COPK.COP_KMeans(
            len(cons.descripList), cons.ml_g, cons.cl_g, f_name)
    else:
        '''---------------- PC-Kmeans ---------------'''
        c_kmeans = PCK.PC_Kmeans(len(cons.descripList), cons.ml_g, cons.cl_g,
                                 cons.neighborhoods, cons.y)

    c_kmeans.fit(cons.x)
    labels = c_kmeans.predict(cons.x)

    print(f'{c_name} Kmeans Clusters:')
    for i in range(len(c_kmeans.clusters)):
        print(f'Cluster {i + 1} :')
        count = 0
        for index in c_kmeans.clusters[i]:
            count += 1
            if count != len(c_kmeans.clusters[i]):
                print(f'{cand_img[index].filename}, ', end='')
            else:
                print(f'{cand_img[index].filename}.')

    return c_kmeans, labels


def gen_feats(f_name, query_img, k, img_classSet, img_dataset, n_imgs):
    '''Generate and Print Constrainted Cluster\n
       Parameters
           f_name: cluster name
           query_img: the query image
           k: k neighbours for kNN
           img_classSet: label class array
           img_dataset: image dataset
           n_imgs: Length of img List
    '''
    if f_name == "BOVW":
        print('----------- Bag of Visual Words -----------')
        t0 = time()
        bovw_param = bovw.BOVW(random_state=random_state,
                               n_patches=n_patches, tam_patch=tam_patch, n_dic=n_dic)
        features = bovw.execute_Bovw(
            img_dataset, bovw_param, n_imgs)
        query_feature = bovw.get_bovw_features(query_img, bovw_param)
        print("BOVW features Creation time: %0.3fs" % (time() - t0))
    elif f_name == "SIFT":
        print('----------- SIFT -----------')
        t0 = time()
        sift_keypoints, features = SIFT.execute_sift(img_dataset, 1000)
        query_feature = SIFT.get_sift_features(query_img)
        print("SIFT features Creation time: %0.3fs" % (time() - t0))
    elif f_name == "MPEG7":
        print('----------- MPEG7 -----------')
        t0 = time()
        features = mpeg7.execute_mpeg7(img_dataset)
        query_feature = mpeg7.get_mpeg7_features(query_img)
        print("MPEG7 features Creation time: %0.3fs" % (time() - t0))

    '''----------- Candidate Selection ---------------'''
    t0 = time()
    cand_img, cand_features = cand_selec.select_candidates(
        f_name, k, features, query_feature, img_dataset, n_imgs)
    print("Candidate Selection time: %0.3fs" % (time() - t0))

    return cand_img, cand_features

In [ ]:
t0 = time()
# Read arguments from command line
args = parser.parse_args(["--k", "50", "--query", "000036"])
k = int(args.k)
query = args.query
path_query = os.getcwd() + img_folder_path + query + '.jpg'
query_img = cv2.imread(path_query)

'''----------- Creating Data Set ------------------'''
image_classSet = dataset.extract_imageDescrip(
    img_class_set_names, img_class_path)
image_dataset, n_imgs = dataset.create_dataset(img_folder_path)
print("Data Set Creation time: %0.3fs" % (time() - t0))


Data Set Creation time: 31.688s


In [18]:
'''Extract image features, candidates using knn & create constraints for given feature name'''
cand_img_mpeg7, cand_features_mpeg7 = gen_feats("MPEG7", query_img, k,
                                     image_classSet, image_dataset, n_imgs)
cand_img_bovw, cand_features_bovw = gen_feats("BOVW", query_img, k,
                                   image_classSet, image_dataset, n_imgs)


----------- MPEG7 -----------
MPEG7 features Creation time: 26.022s
Candidate Selection time: 0.108s
----------- Bag of Visual Words -----------
------------------ Feature Extraction - Bag of Visual Words - START ----------
Patches extracted to create dictionary of features
Total of images =  5011
Size of each array of patches =  (250, 675)
Patches Creation time: 42.631s
Instances =  1252750  size =  10
Created LBP feature spaces
	patches =  1252750  size =  10
LBP Instance Creation time: 74.118s
Kmeans fitting time: 373.672s
Creation of image features: 4.815s
Number of images and features =  (5011, 50)
------------------ Feature Extraction - Bag of Visual Words - END ------------
BOVW features Creation time: 504.488s
Candidate Selection time: 0.088s


In [19]:
'''Extract image features, candidates using knn & create constraints for given feature name'''
cand_img_sift, cand_features_sift = gen_feats("SIFT", query_img, k,
                         image_classSet, image_dataset, n_imgs)


----------- SIFT -----------
SIFT features Creation time: 575.491s
Candidate Selection time: 33.256s


In [20]:
'''----------- Constraint Creation ---------------'''
t0 = time()
cand_img_mpeg7, cons_mpeg7 = gen_cons.generate_constraints(
    cand_img_mpeg7, cand_features_mpeg7, image_classSet)
cand_img_bovw, cons_bovw = gen_cons.generate_constraints(
    cand_img_bovw, cand_features_bovw, image_classSet)
cand_img_sift, cons_sift = gen_cons.generate_constraints(
    cand_img_sift, cand_features_bovw, image_classSet)
print("Constraint Creation time: %0.3fs" % (time() - t0))


Constraint Creation time: 0.256s


In [21]:
'''Generate Clusters'''
mpeg7_copk_clus, mpeg7_copk_labels = gen_clus(
    "COP", cand_img_mpeg7, cons_mpeg7, "MPEG7")
mpeg7_pck_clus, mpeg7_pck_labels = gen_clus(
    "PC", cand_img_mpeg7, cons_mpeg7, "MPEG7")

'''Evaluate Clustering'''
print(
    f'COPKMeans Silhouette Score(n={k}): {clus_eval.silhouette_score("MPEG7", cons_mpeg7.x, mpeg7_copk_labels, len(cons_mpeg7.descripList))}')
print(
    f'PCKMeans Silhouette Score(n={k}): {clus_eval.silhouette_score("MPEG7", cons_mpeg7.x, mpeg7_pck_labels, len(cons_mpeg7.descripList))}')


COP Kmeans Clusters:
Cluster 1 :
007855, 000125, 006162, 006585, 004391, 000382, 000380, 000862, 003657, 002241, 008968, 005897, 006258.
Cluster 2 :
000729, 000460, 008989.
Cluster 3 :
000036, 002938, 009700, 001405, 009144, 006751, 001203, 009717, 005854, 008258, 005805, 005202, 000849, 001348, 002969, 006026, 004537, 002287, 006091, 004368, 003772, 001209.
Cluster 4 :
009524, 008008, 001427, 005315, 002669, 003189, 007122, 003044, 008892, 001686, 003567, 008944.
PC Kmeans Clusters:
Cluster 1 :
000036, 002938, 009700, 001405, 009144, 006751, 001203, 009717, 005854, 008258, 005805, 005202, 000849, 001348, 002969, 006026, 004537, 002287, 006091, 004368, 003772, 001209.
Cluster 2 :
007855, 000125, 006162, 006585, 004391, 000382, 000380, 000862, 003657, 002241, 008968, 005897, 006258.
Cluster 3 :
009524, 008008, 001427, 005315, 002669, 003189, 007122, 003044, 008892, 001686, 003567, 008944.
Cluster 4 :
000729, 000460, 008989.
COPKMeans Silhouette Score(n=50): 0.19780962617910652
PCKMeans 

In [22]:
'''Generate Clusters'''
bovw_copk_clus, bovw_copk_labels = gen_clus(
    "COP", cand_img_bovw, cons_bovw, "BOVW")
bovw_copk_clus, bovw_pck_labels = gen_clus(
    "PC", cand_img_bovw, cons_bovw, "BOVW")

'''Evaluate Clustering'''
print(
    f'COPKMeans Silhouette Score(n={k}): {clus_eval.silhouette_score("BOVW", cons_bovw.x, bovw_copk_labels, len(cons_bovw.descripList))}')
print(
    f'PCKMeans Silhouette Score(n={k}): {clus_eval.silhouette_score("BOVW", cons_bovw.x, bovw_pck_labels, len(cons_bovw.descripList))}')



COP Kmeans Clusters:
Cluster 1 :
005676, 008835, 004682, 001878, 004333, 001158.
Cluster 2 :
000036, 002199, 004264, 000225, 008815, 006495, 005653, 005006, 008773, 006234, 008115, 004634, 009323, 007230, 000682, 005812, 005352, 000359, 003698, 002977, 006295, 005161, 009429, 001192, 006219, 002034, 008722, 003772, 000416.
Cluster 3 :
003170, 009717, 007683, 003645, 004008.
Cluster 4 :
006719, 006371, 006747, 002804, 000871, 005893, 004140, 006736, 000552, 005169.
PC Kmeans Clusters:
Cluster 1 :
000036, 002199, 004264, 000225, 008815, 006495, 005653, 005006, 008773, 006234, 008115, 004634, 009323, 007230, 000682, 005812, 005352, 000359, 003698, 002977, 006295, 005161, 009429, 001192, 006219, 002034, 008722, 003772, 000416.
Cluster 2 :
006719, 006371, 006747, 002804, 000871, 005893, 004140, 006736, 000552, 005169.
Cluster 3 :
005676, 008835, 004682, 001878, 004333, 001158.
Cluster 4 :
003170, 009717, 007683, 003645, 004008.
COPKMeans Silhouette Score(n=50): 0.22599972353197229
PCKMeans 

NameError: name 'sklearn' is not defined

In [ ]:
'''Generate Clusters'''
sift_copk_clus, sift_copk_labels = gen_clus(
    "COP", cand_img_sift, cons_sift, "SIFT")
sift_pck_clus, sift_pck_labels = gen_clus(
    "PC", cand_img_sift, cons_sift, "SIFT")

'''Evaluate Clustering'''
print(
    f'COPKMeans Silhouette Score(n={k}): {clus_eval.silhouette_score("SIFT", cons_sift.x, sift_copk_labels, len(cons_sift.descripList))}')
print(
    f'PCKMeans Silhouette Score(n={k}): {clus_eval.silhouette_score("SIFT", cons_sift.x, sift_pck_labels, len(cons_sift.descripList))}')



error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\core\src\batch_distance.cpp:275: error: (-215:Assertion failed) type == src2.type() && src1.cols == src2.cols && (type == CV_32F || type == CV_8U) in function 'cv::batchDistance'
